In [0]:
%tensorflow_version 2.x # colaboratory requirement

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x # colaboratory requirement`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [0]:
import tensorflow as tf
print(tf.version)

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [0]:
# alg. 1
def filter_ids(data):
    currentId = ''
    idToSet = ''
    for index, row in data.iterrows():
        currentId = str(row['EventId'])
        if currentId == '0':
            data.at[index, 'EventId'] = str(idToSet)
        else:
            currentId = str(row['EventId'])
            idToSet = str(row['EventId'])
    return data


# alg. 2
def filter_events(data):
    """
    Функцията очаква pandas.DataFrame;
    Връща DataFrame съдържащ данни само за ляво и дясно
    """
    data = data[(data.EventId == '33025') | (data.EventId == '33026')]
    return data


In [0]:
import numpy as np
import pandas as pd
from numpy import nan

input_file_url = 'https://github.com/YYx00xZZ/UniBIT-Study/blob/colab/data/input/1_GD_Standart_14ch_24.01.2020.csv?raw=true'
input2_url = 'https://github.com/YYx00xZZ/UniBIT-Study/blob/master/data/input/2_GD_Standart_14ch_24.01.2020.csv?raw=true'
initial_col_names = ['Time128Hz', 'Epoch', 'AF3', 'F7', 'F3', 'F5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4', 'EventId', 'EventDate', 'EventDuration']

In [0]:
import keras

Using TensorFlow backend.


In [0]:
df = pd.read_csv(input_file_url, names=initial_col_names, index_col=False, header=None, sep=',',skiprows=1, dtype={'EventId' : str})
# Replace all NA values with zeros
df.fillna(0, inplace=True)

In [15]:
data = filter_events(filter_ids(df))
# data.head(15)
mapping = {'33025': 0, '33026': 1}
data.replace({'EventId': mapping}, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6666: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regex=regex,


In [16]:
# dframe = data.drop(['EventDate', 'EventDuration'], axis=1)
dframe = dframe.reset_index()
dframe = dframe.drop(['index'], axis=1)

NameError: ignored

In [17]:
data.head()

,Time128Hz,Epoch,AF3,F7,F3,F5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,EventId,EventDate,EventDuration
1771,13.835938,55,4139.487305,4139.487305,4157.692383,4134.743652,4151.282227,4141.922852,4128.974121,4161.666504,4153.077148,4145.128418,4137.692383,4126.666504,4138.077148,4192.436035,0,13.8395803992,0.0000000000
1772,13.843750,55,4139.358887,4136.282227,4150.897461,4123.846191,4141.538574,4145.897461,4135.128418,4159.230957,4156.153809,4153.589844,4144.487305,4126.666504,4138.717773,4187.948730,0,0,0
1773,13.851562,55,4140.897461,4135.128418,4149.615234,4125.897461,4136.538574,4138.333496,4126.282227,4154.743652,4154.358887,4161.538574,4162.692383,4138.077148,4143.974121,4194.871582,0,0,0
1774,13.859375,55,4144.358887,4142.179688,4160.384766,4139.102539,4142.436035,4136.794922,4123.077148,4155.769043,4156.794922,4167.948730,4176.153809,4138.974121,4156.666504,4201.794922,0,0,0
1775,13.867188,55,4143.717773,4144.615234,4167.563965,4137.179688,4141.025879,4140.641113,4123.205078,4158.589844,4156.153809,4166.538574,4175.769043,4138.974121,4166.922852,4206.666504,0,0,0


In [18]:
data.dtypes

Time128Hz        float64
Epoch              int64
AF3              float64
F7               float64
F3               float64
F5               float64
T7               float64
P7               float64
O1               float64
O2               float64
P8               float64
T8               float64
FC6              float64
F4               float64
F8               float64
AF4              float64
EventId            int64
EventDate         object
EventDuration     object
dtype: object

In [0]:
data.to_csv('1GD_Standart.csv', sep=',', index=False)

In [0]:
train_X = dframe.iloc[:, 0:-1]
# train_y = dframe.iloc[:, :15]
train_y = dframe[['EventId']]

In [0]:
from keras.models import Sequential
from keras.layers import Dense
#create model
model = Sequential()

#get number of columns in training data
n_cols = train_X.shape[1]

#add model layers
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

In [0]:
#compile model using mse as a measure of model performance
model.compile(optimizer='adam', loss='mean_squared_error')

In [0]:
from sklearn.model_selection import train_test_split

print('Splitting the data into training and test sets')
X_train, X_test, Y_train, Y_test = train_test_split(train_X, train_y, test_size = 0.2)

Splitting the data into training and test sets


In [0]:
from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)
#train model
model.fit(train_X, train_y, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])

Train on 12176 samples, validate on 3044 samples
Epoch 1/30
12176/12176 [==============================] - 1s 42us/step - loss: 59.8224 - val_loss: 55.5388
Epoch 2/30
12176/12176 [==============================] - 0s 38us/step - loss: 54.3108 - val_loss: 40.9210
Epoch 3/30
12176/12176 [==============================] - 0s 40us/step - loss: 48.2204 - val_loss: 54.2516
Epoch 4/30
12176/12176 [==============================] - 0s 39us/step - loss: 41.8857 - val_loss: 27.8896
Epoch 5/30
12176/12176 [==============================] - 0s 38us/step - loss: 35.7587 - val_loss: 25.1134
Epoch 6/30
12176/12176 [==============================] - 0s 37us/step - loss: 32.8455 - val_loss: 34.1606
Epoch 7/30
12176/12176 [==============================] - 0s 38us/step - loss: 27.3843 - val_loss: 19.4697
Epoch 8/30
12176/12176 [==============================] - 0s 39us/step - loss: 26.6891 - val_loss: 14.8031
Epoch 9/30
12176/12176 [==============================] - 0s 38us/step - loss: 19.2009 - val_lo

In [0]:
#example on how to use our newly trained model on how to make predictions on unseen data (we will pretend our new data is saved in a dataframe called 'test_X').
test_y_predictions = model.predict(X_test)
print(test_y_predictions)
print(X_test)

[[ 0.99700224]
 [-1.2420114 ]
 [-1.635383  ]
 ...
 [-2.3868477 ]
 [-0.64844453]
 [-3.132911  ]]
               AF3           F7  ...           F8          AF4
11364  4294.743652  4557.948730  ...  5036.666504  5267.820312
8428   4082.051270  4099.487305  ...  4116.666504  4114.102539
7751   4123.077148  4192.307617  ...  4120.769043  4096.794922
6115   4117.179688  4111.025879  ...  4133.205078  4182.692383
5631   4089.487061  4139.743652  ...  4064.871826  4110.769043
...            ...          ...  ...          ...          ...
14877  4098.333496  4085.512939  ...  4120.256348  4135.897461
7712   4138.077148  4255.256348  ...  4171.794922  4192.436035
3806   4119.487305  4092.820557  ...  4095.256348  4144.358887
10040  4095.641113  4109.615234  ...  4091.153809  3972.820557
15123  4112.820312  4108.077148  ...  4110.256348  4122.179688

[3044 rows x 14 columns]


asd


In [0]:
from matplotlib import pyplot as plt
%matplotlib inline

time = df_set['Time128Hz']
plt.subplot(2,1,1)
plt.plot(time, side_effects_df['AF3'])
plt.title('Before butterworth Bandpass')

# plt.subplot(2,1,2)
# plt.plot(time, y)
# plt.title('After Butterworth Bandpass')
# plt.xlabel('time (s)')

plt.tight_layout()
plt.show()

In [0]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X = side_effects_df.iloc[:, 0:-1]
Y = side_effects_df.iloc[:, -1]
X = sc.fit_transform(X)
X

In [0]:
from sklearn.model_selection import train_test_split

print('Splitting the data into training and test sets')
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
print('Training set:', X_train.shape)
print('Testing set:', X_test.shape)
print()
print('Training set:', Y_train.shape)
print('Testing set:', Y_test.shape)

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

clf = DecisionTreeClassifier()
print('Training a Decision Tree classifier on the training data')
clf.fit(X_train, Y_train)

In [0]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

dot_data = StringIO()
export_graphviz(clf, out_file=dot_data, filled=True, rounded=True, 
                special_characters=True, feature_names=side_effects_df.columns[:-1])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [0]:
from matplotlib import pyplot as plt
%matplotlib inline

time = df_set['Time128Hz']
plt.subplot(2,1,1)
plt.plot(custom_df['left_right'], custom_df['sumNum'])
plt.title('Before butterworth Bandpass')

plt.subplot(2,1,2)
# plt.plot(time, custom_df['AF3'])
plt.plot(custom_df['left_right'], custom_df['AF3'])
plt.title('After Butterworth Bandpass')
plt.xlabel('time (s)')

plt.tight_layout()
plt.show()

In [0]:
dframe = side_effects_df

In [0]:
print(y.dtype, y.shape)
# X.head(20)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [0]:
print(X_train)

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
print(X_train)

In [0]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=10)
mlp.fit(X_train, y_train.values.ravel())

In [0]:
predictions = mlp.predict(X_test)

In [0]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,predictions))
# print(classification_report(y_test,predictions))